# Import Module

In [ ]:
%load_ext autoreload
import os

import numpy as np

%autoreload 2
import zcu_tools.notebook.single_qubit as zf  # noqa: E402
import zcu_tools.notebook.schedule.v2 as zs  # noqa: E402
from zcu_tools.simulate.fluxonium import FluxoniumPredictor
from zcu_tools.program.v2 import visualize_pulse

# ruff: noqa: I001
from zcu_tools import (  # noqa: E402
    DefaultCfg,
    create_datafolder,
    make_cfg,
    make_sweep,
    save_data,
    make_comment,
)

In [ ]:
import zcu_tools.config as zc

# zc.config.DATA_DRY_RUN = True  # don't save data
# zc.config.YOKO_DRY_RUN = True  # don't run yoko

# Create data folder

In [ ]:
chip_name = r"Q12_2D[2]/Q4"

data_host = None
# data_host = "005-writeboard"

database_path = create_datafolder(os.path.join(os.getcwd(), ".."), prefix=chip_name)

# Connect to zcu216

In [ ]:
from zcu_tools.remote import make_proxy
from zcu_tools.program.base import MyProgram  # noqa: F401
from zcu_tools.tools import get_ip_address  # noqa: F401

zc.config.LOCAL_IP = get_ip_address("Tailscale")
# zc.config.LOCAL_IP = "192.168.10.232"
zc.config.LOCAL_PORT = 8887

soc, soccfg, rm_prog = make_proxy("zcu216-1", 8887, proxy_prog=True)
MyProgram.init_proxy(rm_prog, test=True)
print(soccfg)

In [ ]:
# from myqick import QickSoc  # noqa: E402
# from zcu_tools.tools import get_bitfile

# soc = QickSoc(bitfile=get_bitfile("v2"))
# soccfg = soc
# print(soc)

# Predefine parameters

In [ ]:
# timeFly = 0.45
res_ch = 0
qub_ch = 11
reset_ch = 5
reset_ch2 = 2

DefaultCfg.set_adc(ro_chs=[0])
# DefaultCfg.set_dev(flux_dev="none", flux=0.0)

# Initialize the flux

In [ ]:
from zcu_tools.device import YokoDevControl  # noqa: E402

YokoDevControl.connect_server(
    {
        # "host_ip": data_host,
        "host_ip": "127.0.0.1",
        "dComCfg": {"address": "0x0B21::0x0039::91T810992", "interface": "USB"},
        "outputCfg": {"Current - Sweep rate": 10e-6},
    },
    reinit=True,
)
DefaultCfg.set_dev(flux_dev="yoko")
cur_A = YokoDevControl.get_current()
cur_A

In [ ]:
# cur_A = 0.0e-3
YokoDevControl.set_current(cur_A)
DefaultCfg.set_dev(flux=cur_A)

# Lookback

In [ ]:
exp_cfg = {
    "dac": {
        "res_pulse": {
            "ch": res_ch,
            "nqz": 2,
            # "style": "const",
            # "style": "cosine",
            # "style": "gauss",
            # "sigma": 9.5/4,  # us
            "style": "flat_top",
            "length": 0.65,  # us
            # "raise_pulse": {"style": "gauss", "length": 5.0, "sigma": 0.2},
            "raise_pulse": {"style": "cosine", "length": 0.1},
            "gain": 1.0,
            "freq": 5795.62,
            # "trig_offset": timeFly + 0.05,
            "ro_length": 0.5,  # us
        },
    },
    "adc": {
        "ro_length": 1.5,  # us
        "trig_offset": 0.0,  # us
        # "trig_offset": 0.4,  # us
        "relax_delay": 0.0,  # us
    },
}

In [ ]:
cfg = make_cfg(exp_cfg, rounds=10000)

Ts, signals = zs.measure_lookback(soc, soccfg, cfg, progress=True)

In [ ]:
predict_offset = zf.lookback_show(
    Ts, signals, ratio=0.1, smooth=1.0, pulse_cfg=cfg["dac"]["res_pulse"]
)
predict_offset

In [ ]:
timeFly = float(predict_offset)
timeFly

In [ ]:
filename = "lookback"
save_data(
    filepath=os.path.join(database_path, filename),
    x_info={"name": "Time", "unit": "s", "values": Ts * 1e-6},
    z_info={"name": "Signal", "unit": "a.u.", "values": signals},
    comment=make_comment(cfg, f"adc_trig_offset = {timeFly}us"),
    tag="Lookback",
    server_ip=data_host,
)

# OneTone

In [ ]:
res_name = "R4"

In [ ]:
DefaultCfg.set_pulse(
    probe_rf={
        **exp_cfg["dac"]["res_pulse"],
        "length": 5.1,  # us
        # "trig_offset": 0.0,  # us
        "trig_offset": timeFly + 0.05,
        "ro_length": 5.0,  # us
    }
)
# visualize_pulse(DefaultCfg.get_pulse("probe_rf"), time_fly=timeFly)

## Resonator Frequency

In [ ]:
exp_cfg = {
    "dac": {
        # "res_pulse": "readout_rf",
        "res_pulse": {
            **DefaultCfg.get_pulse("probe_rf"),
            "ch": res_ch,
            "nqz": 2,
            "gain": 0.2,
        },
        # "reset": "pulse",
        # "reset_pulse": "reset_red",
        # "reset_pulse": "pi_amp"
        "reset": "mux_dual_pulse",
        "reset_pulse1": "mux_reset1",
        "reset_pulse2": "mux_reset2",
        "reset_pi_pulse": "pi_amp",
    },
    "adc": {
        "relax_delay": 0.0,  # us
        # "relax_delay": 5*t1,  # us
    },
}

In [ ]:
# exp_cfg["sweep"] = make_sweep(5770, 5820, 101)
exp_cfg["sweep"] = make_sweep(r_f-3, r_f+3, 31)
cfg = make_cfg(exp_cfg, reps=1000, rounds=10)

fpts, signals = zs.measure_res_freq(soc, soccfg, cfg)

In [ ]:
f, kappa = zf.freq_analyze(fpts, signals, asym=True)
f

In [ ]:
r_f = f
rf_w = kappa

In [ ]:
filename = f"{res_name}_freq@{cur_A * 1e3:.3f}mA"
save_data(
    filepath=os.path.join(database_path, filename),
    x_info={"name": "Frequency", "unit": "Hz", "values": fpts * 1e6},
    z_info={"name": "Signal", "unit": "a.u.", "values": signals},
    comment=make_comment(cfg, f"resonator frequency = {r_f}MHz"),
    # comment=make_comment(cfg),
    tag="OneTone/single",
    server_ip=data_host,
)

In [ ]:
# r_f =  6026.1
r_f

## Onetone Dependences

### Power Dependence

In [ ]:
exp_cfg = {
    "dac": {
        "res_pulse": {
            **DefaultCfg.get_pulse("probe_rf"),
            "ch": res_ch,
            "nqz": 2,
            "length": 5.1,
            "ro_length": 5.0,
        },
        # "reset": "pulse",
        # "reset_pulse": "reset_red",
        # "reset": "mux_dual_pulse",
        # "reset_pulse1": "mux_reset1",
        # "reset_pulse2": "mux_reset2",
    },
    "adc": {
        "relax_delay": 1.0,  # us
    },
}

In [ ]:
exp_cfg["sweep"] = {
    "gain": make_sweep(0.01, 0.5, 51),
    # "gain": make_sweep(10, 30000, 30, force_int=True),
    # "freq": make_sweep(7522.5, 7535, 100),
    "freq": make_sweep(r_f - 7, r_f + 7, 51),
    # "freq": make_sweep(7510, 7528, 101)
}
cfg = make_cfg(exp_cfg, reps=1000, rounds=100)

pdrs, fpts, signals2D = zs.measure_res_pdr_dep(
    soc, soccfg, cfg, dynamic_avg=True, gain_ref=0.05
)

In [ ]:
filename = f"{res_name}_pdr@{cur_A * 1e3:.3f}mA"
save_data(
    filepath=os.path.join(database_path, filename),
    x_info={"name": "Frequency", "unit": "Hz", "values": fpts * 1e6},
    y_info={"name": "Power", "unit": "a.u.", "values": pdrs},
    z_info={"name": "Signal", "unit": "a.u.", "values": signals2D},
    comment=make_comment(cfg),
    tag="OneTone/pdr",
    server_ip=data_host,
)

### Flux dependence

In [ ]:
cur_A = -1.5e-3
YokoDevControl.set_current(cur_A)
DefaultCfg.set_dev(flux=cur_A)
cur_A * 1e3

In [ ]:
exp_cfg = {
    "dac": {
        "res_pulse": {
            **DefaultCfg.get_pulse("probe_rf"),
            "ch": res_ch,
            "nqz": 2,
            "gain": 0.3,
            # "gain": 30000,
            "length": 5.1,
            "ro_length": 5.0,
        },
    },
    "adc": {
        "relax_delay": 0.0,  # us
    },
}

In [ ]:
exp_cfg["sweep"] = {
    "flux": make_sweep(-1.5e-3, 3.5e-3, 151),
    "freq": make_sweep(r_f - 7, r_f + 7, 101),
    # "freq": make_sweep(7518.0, 7525.0, 101),
}
cfg = make_cfg(exp_cfg, reps=100, rounds=10)

As, fpts, signals2D = zs.measure_res_flux_dep(soc, soccfg, cfg)

In [ ]:
filename = f"{res_name}_flux"
save_data(
    filepath=os.path.join(database_path, filename),
    x_info={"name": "Frequency", "unit": "Hz", "values": fpts * 1e6},
    y_info={"name": "Current", "unit": "A", "values": As},
    z_info={"name": "Signal", "unit": "a.u.", "values": signals2D},
    comment=make_comment(cfg),
    tag="OneTone/flux",
    server_ip=data_host,
)

In [ ]:
cur_A = 2.184e-3
YokoDevControl.set_current(cur_A)
DefaultCfg.set_dev(flux=cur_A)

## Set readout pulse

In [ ]:
# r_f = 7000.0
DefaultCfg.set_pulse(
    readout_rf={
        **DefaultCfg.get_pulse("probe_rf"),
        "ch": res_ch,
        "nqz": 2,
        "freq": r_f,  # MHz
        "gain": 0.2,
        "length": 2.6,
        "ro_length": 2.5,
        "pre_delay": 0.0,
        "desc": "lower power readout with exact resonator frequency",
    },
)
# visualize_pulse(DefaultCfg.get_pulse("readout_rf"), time_fly=timeFly)

# TwoTone

In [ ]:
qub_name = "Q4"

In [ ]:
preditor = FluxoniumPredictor(f"../result/{chip_name}/params.json")

In [ ]:
DefaultCfg.set_pulse(
    probe_qf={
        "style": "flat_top",
        "length": 2.0,
        # "raise_pulse": {"style": "gauss", "length": 0.02, "sigma": 0.003},
        "raise_pulse": {"style": "cosine", "length": 0.02},
    },
)
# visualize_pulse(DefaultCfg.get_pulse("probe_qf"))

## Twotone Frequency

In [ ]:
# cur_A = -0.4145e-3
YokoDevControl.set_current(cur_A)
DefaultCfg.set_dev(flux=cur_A)
cur_A * 1e3

In [ ]:
q_f = preditor.predict_freq(cur_A, transition=(0, 1))
q_f

In [ ]:
exp_cfg = {
    "dac": {
        "res_pulse": "readout_rf",
        # "res_pulse": "readout_dpm",
        "qub_pulse": {
            **DefaultCfg.get_pulse("probe_qf"),
            "ch": qub_ch,
            "nqz": 1,
            "gain": 0.1,
            "length": 0.5,  # us
            "mixer_freq": q_f,
        },
        # "qub_pulse": "pi_amp",
        # "reset": "pulse",
        # "reset_pulse": "reset_red",
        "reset": "mux_dual_pulse",
        "reset_pulse1": "mux_reset1",
        "reset_pulse2": "mux_reset2",
    },
    "adc": {
        "relax_delay": 0.0,  # us
        # "relax_delay": 5*t1,  # us
    },
}

In [ ]:
# exp_cfg["sweep"] = make_sweep(q_f - 20, q_f + 20, 101)
exp_cfg["sweep"] = make_sweep(325, 375, 101)
cfg = make_cfg(exp_cfg, reps=1000, rounds=10)

# zs.visualize_qub_freq(soccfg, cfg, time_fly=timeFly)
fpts, signals = zs.measure_qub_freq(soc, soccfg, cfg, remove_bg=True)

In [ ]:
%matplotlib inline
f, kappa = zf.freq_analyze(fpts, signals, max_contrast=True)
f

In [ ]:
q_f = f
qf_w = kappa

In [ ]:
filename = f"{qub_name}_freq@{cur_A * 1e3:.3f}mA"
save_data(
    filepath=os.path.join(database_path, filename),
    x_info={"name": "Frequency", "unit": "Hz", "values": fpts * 1e6},
    z_info={"name": "Signal", "unit": "a.u.", "values": signals},
    comment=make_comment(cfg, f"frequency = {f}MHz"),
    tag="TwoTone/single",
    server_ip=data_host,
)

In [ ]:
bias = preditor.calculate_bias(cur_A, q_f)
bias * 1e3

In [ ]:
preditor.update_bias(bias)

## Reset

### One Pulse

In [ ]:
reset_f = r_f - q_f
reset_f

In [ ]:
exp_cfg = {
    "dac": {
        # "res_pulse": "readout_rf",
        "res_pulse": "readout_dpm",
        "qub_pulse": {
            **DefaultCfg.get_pulse("probe_qf"),
            "ch": reset_ch,
            "nqz": 2,
            "gain": 0.5,
            "length": 5,  # us
            "mixer_freq": reset_f,
        },
        # "reset": "pulse",
        # "reset_pulse": {
        #     **DefaultCfg.get_pulse("probe_qf"),
        #     "ch": qub_ch,
        #     "nqz": 1,
        #     "gain": 0.07,
        #     "length": 2,  # us
        #     "freq": q_f,
        #     "mixer_freq": q_f,
        # },
    },
    "adc": {
        "relax_delay": 0.0,  # us
        # "relax_delay": 5*t1
    },
}

In [ ]:
exp_cfg["sweep"] = make_sweep(reset_f - 150, reset_f + 150, 101)
# exp_cfg["sweep"] = make_sweep(5360, 5450, 101)
cfg = make_cfg(exp_cfg, reps=1000, rounds=10)

fpts, signals = zs.measure_qub_freq(soc, soccfg, cfg, remove_bg=True)

In [ ]:
f, kappa = zf.freq_analyze(fpts, signals, max_contrast=True)
f

In [ ]:
reset_f = f

In [ ]:
filename = f"{qub_name}_freq@{cur_A * 1e3:.3f}mA"
save_data(
    filepath=os.path.join(database_path, filename),
    x_info={"name": "Frequency", "unit": "Hz", "values": fpts * 1e6},
    z_info={"name": "Signal", "unit": "a.u.", "values": signals},
    comment=make_comment(cfg, f"frequency = {f}MHz"),
    tag="TwoTone/single",
    server_ip=data_host,
)

#### Reset Time

In [ ]:
exp_cfg = {
    "dac": {
        # "res_pulse": "readout_rf",
        "res_pulse": "readout_dpm",
        "qub_pulse": {
            **DefaultCfg.get_pulse("pi_amp"),
        },
        "reset": "pulse",
        "reset_pulse": {
            **DefaultCfg.get_pulse("probe_qf"),
            "ch": reset_ch,
            "nqz": 2,
            "gain": 0.5,
            "freq": reset_f,
            "mixer_freq": reset_f,
        },
    },
    "adc": {
        "relax_delay": 0.0,  # us
    },
}

In [ ]:
exp_cfg["sweep"] = make_sweep(0.03, 5.0, 51)
cfg = make_cfg(exp_cfg, reps=1000, rounds=100)

# zs.visualize_reset_time(soccfg, cfg, time_fly=timeFly)
Ts, signals = zs.measure_reset_time(soc, soccfg, cfg)

In [ ]:
filename = f"{qub_name}_reset_time@{cur_A * 1e3:.3f}mA"
save_data(
    filepath=os.path.join(database_path, filename),
    x_info={"name": "Length", "unit": "s", "values": Ts * 1e-6},
    z_info={"name": "Signal", "unit": "a.u.", "values": signals},
    comment=make_comment(cfg),
    tag="TwoTone/reset/time",
    server_ip=data_host,
)

#### Set Reset Pulse

In [ ]:
DefaultCfg.set_pulse(
    reset_red={
        **exp_cfg["dac"]["reset_pulse"],
        "length": 0.4,  # us
        "post_delay": 5.0 / rf_w,  # 5 times the resonator linewidth
        "desc": "Reset pulse by red sideband",
    },
)
visualize_pulse(DefaultCfg.get_pulse("reset_red"))

### Two pulse

In [ ]:
reset1_trans = (1, 2)
reset_f1 = preditor.predict_freq(cur_A, transition=reset1_trans)
reset_f1

In [ ]:
reset2_trans = (2, 0)
reset_f2 = abs(r_f + preditor.predict_freq(cur_A, transition=reset2_trans))
reset_f2

In [ ]:
exp_cfg = {
    "dac": {
        "res_pulse": "readout_rf",
        # "res_pulse": "readout_dpm",
        "reset": "mux_dual_pulse",
        "reset_pulse1": {
            **DefaultCfg.get_pulse("probe_qf"),
            "ch": reset_ch,
            "nqz": 2,
            "gain": 1.0,
            "length": 5,  # us
            "mixer_freq": reset_f1,
        },
        "reset_pulse2": {
            **DefaultCfg.get_pulse("probe_qf"),
            "ch": reset_ch2,
            "nqz": 1,
            "gain": 1.0,
            "length": 5,  # us
            # "mixer_freq": reset_f2,
        },
    },
    "adc": {
        "relax_delay": 0.0,  # us
        # "relax_delay": t1,  # us
    },
}

In [ ]:
exp_cfg["sweep"] = {
    # "freq1": make_sweep(reset_f1 - 100, reset_f1 + 100, 51),
    # "freq2": make_sweep(reset_f2 - 400, reset_f2 + 100, 151),
    "freq1": make_sweep(3150, 3270, 51),
    "freq2": make_sweep(1900, 2050, 51),
}
cfg = make_cfg(exp_cfg, reps=100, rounds=100)

fpts1, fpts2, signals = zs.measure_mux_reset_freq(soc, soccfg, cfg)

In [ ]:
%matplotlib inline
xlabal = f"|{reset1_trans[0]}, 0> - |{reset1_trans[1]}, 0>"
ylabal = f"|{reset2_trans[0]}, 0> - |{reset2_trans[1]}, 1>"
f1, f2 = zf.mux_reset_analyze(
    fpts1, fpts2, signals, xlabel=xlabal, ylabel=ylabal, smooth=0.7
)
f1, f2

In [ ]:
reset_f1 = f1
reset_f2 = f2

In [ ]:
filename = f"{qub_name}_mux_reset_freq@{cur_A * 1e3:.3f}mA"
save_data(
    filepath=os.path.join(database_path, filename),
    x_info={"name": xlabal, "unit": "Hz", "values": fpts1 * 1e6},
    y_info={"name": ylabal, "unit": "Hz", "values": fpts2 * 1e6},
    z_info={"name": "Signal", "unit": "a.u.", "values": signals.T},
    comment=make_comment(cfg, f"frequency = ({reset_f1:.1f}, {reset_f2:.1f})MHz"),
    tag="TwoTone/mux_reset",
    server_ip=data_host,
)

#### Reset Time

In [ ]:
exp_cfg = {
    "dac": {
        "res_pulse": "readout_rf",
        # "res_pulse": "readout_dpm",
        "qub_pulse": "pi_amp",
        "reset": "mux_dual_pulse",
        "reset_pulse1": {
            **DefaultCfg.get_pulse("probe_qf"),
            "ch": reset_ch,
            "nqz": 2,
            "gain": 1.0,
            "freq": reset_f1,
            "mixer_freq": reset_f1,
        },
        "reset_pulse2": {
            **DefaultCfg.get_pulse("probe_qf"),
            "ch": reset_ch2,
            "nqz": 1,
            "gain": 1.0,
            "freq": reset_f2,
            # "mixer_freq": reset_f2,
        },
    },
    "adc": {
        "relax_delay": 2.0,  # us
        # "relax_delay": t1,  # us
    },
}

In [ ]:
exp_cfg["sweep"] = make_sweep(1.0, 10.0, 31)
cfg = make_cfg(exp_cfg, reps=1000, rounds=100)

# zs.visualize_reset_time(soccfg, cfg, time_fly=timeFly)
Ts, signals = zs.measure_reset_time(soc, soccfg, cfg)

In [ ]:
filename = f"{qub_name}_mux_reset_time@{cur_A * 1e3:.3f}mA"
save_data(
    filepath=os.path.join(database_path, filename),
    x_info={"name": "Length", "unit": "s", "values": Ts * 1e-6},
    z_info={"name": "Signal", "unit": "a.u.", "values": signals},
    comment=make_comment(cfg),
    tag="TwoTone/mux_reset/time",
    server_ip=data_host,
)

#### Set Mux Reset Pulse

In [ ]:
mux_reset_len = 6.0
DefaultCfg.set_pulse(
    mux_reset1={
        **exp_cfg["dac"]["reset_pulse1"],
        "freq": reset_f1,
        "length": mux_reset_len,  # us
        "desc": "Reset pulse by red sideband",
    },
    mux_reset2={
        **exp_cfg["dac"]["reset_pulse2"],
        "freq": reset_f2,
        "length": mux_reset_len,  # us
        "post_delay": 5.0 / rf_w,  # 5 times the resonator linewidth
        "desc": "Reset pulse by red sideband",
    },
)

### Check Reset

In [ ]:
exp_cfg = {
    "dac": {
        # "res_pulse": "readout_rf",
        "res_pulse": "readout_dpm",
        "qub_pulse": "pi_amp",
        # "reset_test": "pulse",
        # "reset_test_pulse": "reset_red",
        "reset_test": "mux_dual_pulse",
        "reset_test_pulse1": "mux_reset1",
        "reset_test_pulse2": "mux_reset2",
        "reset": "mux_dual_pulse",
        "reset_pulse1": "mux_reset1",
        "reset_pulse2": "mux_reset2",
    },
    "adc": {
        "relax_delay": 0.0,  # us
        # "relax_delay": 3*t1,  # us
    },
}

In [ ]:
# exp_cfg["sweep"] = make_sweep(0.0, 0.5, 51)
exp_cfg["sweep"] = make_sweep(0.0, max_gain, 51)
cfg = make_cfg(exp_cfg, reps=1000, rounds=10)

# zs.visualize_reset_amprabi(soccfg, cfg, time_fly=timeFly)
pdrs, signals = zs.measure_reset_amprabi(soc, soccfg, cfg)

## Twotone Dependences

### Power dependence

In [ ]:
exp_cfg = {
    "dac": {
        "res_pulse": "readout_rf",
        "qub_pulse": {
            **DefaultCfg.get_pulse("probe_qf"),
            "ch": qub_ch,
            "nqz": 2,
            "length": 5,  # us
        },
    },
    "adc": {
        "relax_delay": 0.0,  # us
    },
}

In [ ]:
exp_cfg["sweep"] = {
    "gain": make_sweep(0.05, 1.0, 30),
    "freq": make_sweep(1700, 2000, 30),
}
cfg = make_cfg(exp_cfg, reps=50, rounds=50)

fpts, pdrs, signals2D = zs.measure_qub_pdr_dep(soc, soccfg, cfg)

In [ ]:
filename = f"{qub_name}_pdr@{cur_A * 1e3:.3f}mA"
save_data(
    filepath=os.path.join(database_path, filename),
    x_info={"name": "Frequency", "unit": "Hz", "values": fpts * 1e6},
    y_info={"name": "Power", "unit": "a.u.", "values": pdrs},
    z_info={"name": "Signal", "unit": "a.u.", "values": signals2D},
    comment=make_comment(cfg),
    tag="TwoTone/pdr",
    server_ip=data_host,
)

### Flux Dependence

In [ ]:
cur_A = -1.25e-3
YokoDevControl.set_current(cur_A)
DefaultCfg.set_dev(flux=cur_A)

In [ ]:
exp_cfg = {
    "dac": {
        "res_pulse": {
            **DefaultCfg.get_pulse("probe_rf"),
            "ch": res_ch,
            "nqz": 2,
            "freq": r_f,  # MHz
            "gain": 0.2,
            "length": 1.7,
            "ro_length": 1.5,
        },
        # "res_pulse": "readout_dpm",
        "qub_pulse": {
            **DefaultCfg.get_pulse("probe_qf"),
            "ch": qub_ch,
            "nqz": 1,
            "gain": 0.2,
            "length": 2,  # us
        },
        # "reset": "pulse",
        # "reset_pulse": "reset_red",
    },
    "adc": {
        "relax_delay": 0.5,  # us
    },
}

In [ ]:
exp_cfg["sweep"] = {
    "flux": make_sweep(-1.25e-3, -3e-3, 151),
    "freq": make_sweep(150, 3250, 1201),
}
cfg = make_cfg(exp_cfg, reps=10000, rounds=10)

As, fpts, signals2D = zs.measure_qub_flux_dep(soc, soccfg, cfg)

In [ ]:
filename = f"{qub_name}_flux_dep"
save_data(
    filepath=os.path.join(database_path, filename),
    y_info={"name": "Current", "unit": "A", "values": As},
    x_info={"name": "Frequency", "unit": "Hz", "values": fpts * 1e6},
    z_info={"name": "Signal", "unit": "a.u.", "values": signals2D},
    comment=make_comment(cfg),
    tag="TwoTone/flux",
    server_ip=data_host,
)

In [ ]:
cur_A = -3e-3
YokoDevControl.set_current(cur_A)
DefaultCfg.set_dev(flux=cur_A)

## AC Stark Shift

In [ ]:
exp_cfg = {
    "dac": {
        "res_pulse": "readout_dpm",
        "stark_res_pulse": {
            **DefaultCfg.get_pulse("readout_dpm"),
            "length": 1.1,
            "ro_length": None,
            "trig_offset": None,
        },
        "stark_qub_pulse": {
            **DefaultCfg.get_pulse("pi_amp"),
            # "length": 3,
        },
        "reset": "mux_dual_pulse",
        "reset_pulse1": "mux_reset1",
        "reset_pulse2": "mux_reset2",
    },
    "adc": {
        "relax_delay": 0.0,  # us
    },
}

In [ ]:
exp_cfg["sweep"] = {
    "gain": make_sweep(0.00, 0.05, 51),
    # "freq": make_sweep(q_f - 100, q_f + 100, 101),
    "freq": make_sweep(q_f - 20, q_f + 20, 101),
}
cfg = make_cfg(exp_cfg, reps=100, rounds=100)

# zs.visualize_ac_stark(soccfg, cfg, time_fly=timeFly)
pdrs, fpts, signals2D = zs.measure_ac_stark(soc, soccfg, cfg)

In [ ]:
filename = f"{qub_name}_ac_stark@{cur_A * 1e3:.3f}mA"
save_data(
    filepath=os.path.join(database_path, filename),
    x_info={"name": "Power", "unit": "a.u.", "values": pdrs},
    y_info={"name": "Frequency", "unit": "Hz", "values": fpts * 1e6},
    z_info={"name": "Signal", "unit": "a.u.", "values": signals2D},
    comment=make_comment(cfg),
    tag="TwoTone/ac_stark",
    server_ip=data_host,
)

# Rabi

## Length Rabi

In [ ]:
exp_cfg = {
    "dac": {
        # "res_pulse": "readout_rf",
        "res_pulse": "readout_dpm",
        "qub_pulse": {
            **DefaultCfg.get_pulse("probe_qf"),
            "ch": qub_ch,
            "nqz": 1,
            "freq": q_f,
            "gain": 0.3,
            # "gain": pi_gain,
            "mixer_freq": q_f,
        },
        # "reset": "pulse",
        # "reset_pulse": "reset_red",
        "reset": "mux_dual_pulse",
        "reset_pulse1": "mux_reset1",
        "reset_pulse2": "mux_reset2",
    },
    "adc": {
        "relax_delay": 0.0,  # us
        # "relax_delay": 3*t1,  # us
    },
}

In [ ]:
exp_cfg["sweep"] = make_sweep(0.03, 0.3, 101)
# exp_cfg["sweep"] = make_sweep(0.06, 5 * pi_len, 101)
cfg = make_cfg(exp_cfg, reps=1000, rounds=20)

# zs.visualize_lenrabi(soccfg, cfg, force_align=False, time_fly=timeFly)
Ts, signals = zs.measure_lenrabi(soc, soccfg, cfg, force_align=False)

In [ ]:
%matplotlib inline
pi_len, pi2_len = zf.rabi_analyze(
    Ts, signals, decay=False, max_contrast=True, xlabel="Time (us)"
)
pi_len, pi2_len

In [ ]:
filename = f"{qub_name}_len_rabi@{cur_A * 1e3:.3f}mA"
save_data(
    filepath=os.path.join(database_path, filename),
    x_info={"name": "Pulse Length", "unit": "s", "values": Ts * 1e-6},
    z_info={"name": "Signal", "unit": "a.u.", "values": signals},
    comment=make_comment(cfg, f"pi len = {pi_len}us\npi/2 len = {pi2_len}us"),
    tag="TimeDomain/len_rabi",
    server_ip=data_host,
)

### Set Pi Pulse

In [ ]:
# pi_len = 1.0
# pi2_len = 0.5

In [ ]:
DefaultCfg.set_pulse(
    pi_len={
        **cfg["dac"]["qub_pulse"],
        "length": pi_len,
        "desc": "len pi pulse",
    },
    pi2_len={
        **cfg["dac"]["qub_pulse"],
        "length": pi2_len,
        "desc": "len pi/2 pulse",
    },
)
visualize_pulse([DefaultCfg.get_pulse("pi_len"), DefaultCfg.get_pulse("pi2_len")])

## Amplitude Rabi

In [ ]:
# pi_gain = 0.1
max_gain = min(5 * pi_gain, 1.0)
exp_cfg = {
    "dac": {
        # "res_pulse": "readout_rf",
        "res_pulse": "readout_dpm",
        "qub_pulse": {
            **DefaultCfg.get_pulse("probe_qf"),
            "ch": qub_ch,
            "nqz": 1,
            "freq": q_f,
            # "length": 5.0,
            # "length": 3 * pi_len,
            "length": 5 * pi_len * pi_gain / max_gain,
            "mixer_freq": q_f,
        },
        # "reset": "pulse",
        # "reset_pulse": "reset_red",
        "reset": "mux_dual_pulse",
        "reset_pulse1": "mux_reset1",
        "reset_pulse2": "mux_reset2",
    },
    "adc": {
        "relax_delay": 0.0,  # us
        # "relax_delay": 3*t1,  # us
    },
}

In [ ]:
# exp_cfg["sweep"] = make_sweep(0.0, 1.0, 51)
exp_cfg["sweep"] = make_sweep(0.0, max_gain, 51)
cfg = make_cfg(exp_cfg, reps=1000, rounds=10)

# zs.visualize_amprabi(soccfg, cfg, time_fly=timeFly)
pdrs, signals = zs.measure_amprabi(soc, soccfg, cfg)

In [ ]:
%matplotlib inline
pi_gain, pi2_gain = zf.rabi_analyze(
    pdrs, signals, decay=True, max_contrast=True, xlabel="Power (a.u.)"
)
pi_gain = int(pi_gain + 0.5) if pi_gain > 1.0 else pi_gain
pi2_gain = int(pi2_gain + 0.5) if pi2_gain > 1.0 else pi2_gain
pi_gain, pi2_gain

In [ ]:
filename = f"{qub_name}_amp_rabi@{cur_A * 1e3:.3f}mA"
save_data(
    filepath=os.path.join(database_path, filename),
    x_info={"name": "Amplitude", "unit": "a.u.", "values": pdrs},
    z_info={"name": "Signal", "unit": "a.u.", "values": signals},
    comment=make_comment(cfg, f"pi gain = {pi_gain}\npi/2 gain = {pi2_gain}"),
    tag="TimeDomain/amp_rabi",
    server_ip=data_host,
)

### Set Pi Pulse

In [ ]:
# pi_gain = 1.0
# pi2_gain = 0.5

In [ ]:
DefaultCfg.set_pulse(
    pi_amp={
        **cfg["dac"]["qub_pulse"],
        "gain": pi_gain,
        "desc": "amp pi pulse",
    },
    pi2_amp={
        **cfg["dac"]["qub_pulse"],
        "gain": pi2_gain,
        "desc": "amp pi/2 pulse",
    },
)
visualize_pulse([DefaultCfg.get_pulse("pi_amp"), DefaultCfg.get_pulse("pi2_amp")])


# Optimize Readout

## Frequency tuning

In [ ]:
exp_cfg = {
    "dac": {
        "res_pulse": {
            **DefaultCfg.get_pulse("readout_rf"),
            # "freq": fpt_max,
            # "gain": pdr_max,
        },
        # "qub_pulse": "pi_len",
        "qub_pulse": "pi_amp",
        # "qub_pulse": "reset_red",
        # "qub_pulse": {
        #     **DefaultCfg.get_pulse('probe_qf'),
        #     "ch": qub_ch,
        #     "nqz": 1,
        #     "freq": q_f,
        #     "gain": 0.02,
        #     "length": 2,  # us
        # },
        # "reset": "pulse",
        # "reset_pulse": "reset_red",
        "reset": "mux_dual_pulse",
        "reset_pulse1": "mux_reset1",
        "reset_pulse2": "mux_reset2",
    },
    "adc": {
        "relax_delay": 0.1,  # us
        # "relax_delay": t1
    },
}

In [ ]:
exp_cfg["sweep"] = make_sweep(r_f - 10, r_f + 10, 51)
cfg = make_cfg(exp_cfg, reps=1000, rounds=10)

fpts, snrs = zs.qubit.measure_ge_freq_dep(soc, soccfg, cfg)

In [ ]:
fpt_max = zf.dispersive1D_analyze(fpts, snrs, xlabel="Frequency (MHz)")

In [ ]:
filename = f"{qub_name}_ge_fpt@{cur_A * 1e3:.3f}mA"
save_data(
    filepath=os.path.join(database_path, filename),
    x_info={"name": "Frequency", "unit": "Hz", "values": fpts * 1e6},
    z_info={"name": "SNR", "unit": "a.u.", "values": snrs},
    comment=make_comment(cfg),
    tag="TwoTone/dispersive/fpt",
    server_ip=data_host,
)

## Power tuning

In [ ]:
exp_cfg = {
    "dac": {
        "res_pulse": {
            **DefaultCfg.get_pulse("readout_rf"),
            "freq": fpt_max,
            # "gain": pdr_max,
        },
        # "qub_pulse": "pi_len",
        "qub_pulse": "pi_amp",
        # "qub_pulse": "reset_red",
        # "qub_pulse": {
        #     **DefaultCfg.get_pulse('probe_qf'),
        #     "ch": qub_ch,
        #     "nqz": 1,
        #     "freq": q_f,
        #     "gain": 0.02,
        #     "length": 2,  # us
        # },
        # "reset": "pulse",
        # "reset_pulse": "reset_red",
        "reset": "mux_dual_pulse",
        "reset_pulse1": "mux_reset1",
        "reset_pulse2": "mux_reset2",
    },
    "adc": {
        "relax_delay": 0.0,  # us
        # "relax_delay": t1
    },
}

In [ ]:
exp_cfg["sweep"] = make_sweep(0.01, 1.0, 31)
cfg = make_cfg(exp_cfg, reps=1000, rounds=10)

pdrs, snrs = zs.qubit.measure_ge_pdr_dep(soc, soccfg, cfg)

In [ ]:
pdr_max = zf.dispersive1D_analyze(pdrs, snrs, xlabel="Probe Power (a.u)")

In [ ]:
filename = f"{qub_name}_ge_pdr@{cur_A * 1e3:.3f}mA"
save_data(
    filepath=os.path.join(database_path, filename),
    x_info={"name": "Probe Power (a.u)", "unit": "s", "values": pdrs},
    z_info={"name": "SNR", "unit": "a.u.", "values": snrs},
    comment=make_comment(cfg),
    tag="TwoTone/dispersive/pdr",
    server_ip=data_host,
)

## Readout Length tuning

In [ ]:
exp_cfg = {
    "dac": {
        "res_pulse": {
            **DefaultCfg.get_pulse("readout_rf"),
            "freq": fpt_max,
            "gain": pdr_max,
        },
        # "qub_pulse": "pi_len",
        "qub_pulse": "pi_amp",
        # "qub_pulse": "reset_red",
        # "qub_pulse": {
        #     **DefaultCfg.get_pulse('probe_qf'),
        #     "ch": qub_ch,
        #     "nqz": 1,
        #     "freq": q_f,
        #     "gain": 0.02,
        #     "length": 2,  # us
        # },
        # "reset": "pulse",
        # "reset_pulse": "reset_red",
        "reset": "mux_dual_pulse",
        "reset_pulse1": "mux_reset1",
        "reset_pulse2": "mux_reset2",
    },
    "adc": {
        "relax_delay": 0.0,  # us
        # "relax_delay": t1
    },
}

In [ ]:
exp_cfg["sweep"] = make_sweep(0.1, 15.0, 31)
cfg = make_cfg(exp_cfg, reps=10000, rounds=1)

ro_lens, snrs = zs.qubit.measure_ge_ro_dep(soc, soccfg, cfg)

In [ ]:
ro_max = zf.dispersive_ro_len_analyze(ro_lens, snrs, t0=30.0)
ro_max

In [ ]:
# ro_max = 3.0

In [ ]:
filename = f"{qub_name}_ge_ro@{cur_A * 1e3:.3f}mA"
save_data(
    filepath=os.path.join(database_path, filename),
    x_info={"name": "Readout length", "unit": "s", "values": ro_lens * 1e-6},
    z_info={"name": "SNR", "unit": "a.u.", "values": snrs},
    comment=make_comment(cfg),
    tag="TwoTone/dispersive/ro_len",
    server_ip=data_host,
)

In [ ]:
DefaultCfg.set_pulse(
    readout_dpm={
        **DefaultCfg.get_pulse("readout_rf"),
        "freq": fpt_max,
        "gain": pdr_max,
        "length": ro_max + 0.2,
        "ro_length": ro_max,
        "pre_delay": 0.0,
        "desc": "Readout with largest dispersive shift",
    },
)
visualize_pulse(DefaultCfg.get_pulse("readout_dpm"), time_fly=timeFly)

# T1 & T2

## T2Ramsey

In [ ]:
orig_qf = DefaultCfg.get_pulse("pi_amp")["freq"]
orig_qf

In [ ]:
exp_cfg = {
    "dac": {
        "res_pulse": "readout_dpm",
        "qub_pulse": "pi2_amp",
        # "reset": "pulse",
        # "reset_pulse": "reset_red",
        "reset": "mux_dual_pulse",
        "reset_pulse1": "mux_reset1",
        "reset_pulse2": "mux_reset2",
    },
    "adc": {
        "relax_delay": 0.5,  # us
        # "relax_delay": 3*t1,  # us
    },
}

In [ ]:
# exp_cfg["sweep"] = make_sweep(0, 15.0, 101)  # us
exp_cfg["sweep"] = make_sweep(0, 3 * t2r, 101)  # us
cfg = make_cfg(exp_cfg, reps=1000, rounds=10)

activate_detune = 20.0 / (cfg["sweep"]["stop"] - cfg["sweep"]["start"])
print(f"activate_detune: {activate_detune:.2f}")
# zs.visualize_t2ramsey(soccfg, cfg, detune=activate_detune, time_fly=timeFly)
Ts, signals = zs.measure_t2ramsey(soc, soccfg, cfg, detune=activate_detune)

In [ ]:
%matplotlib inline
t2r, detune, _, _ = zf.T2fringe_analyze(Ts, signals, max_contrast=True)
# t2d, _ = zf.T2decay_analyze(Ts, signals, max_contrast=True)
print(f"real detune: {(detune-activate_detune)*1e3:.1f}kHz")

In [ ]:
filename = f"{qub_name}_t2ramsey@{cur_A * 1e3:.3f}mA"
save_data(
    filepath=os.path.join(database_path, filename),
    x_info={"name": "Time", "unit": "s", "values": Ts * 1e-6},
    z_info={"name": "Signal", "unit": "a.u.", "values": signals},
    comment=make_comment(cfg, f"detune = {detune}MHz\nt2r = {t2r}us"),
    tag="TimeDomain/t2ramsey",
    server_ip=data_host,
)

In [ ]:
q_f = orig_qf + activate_detune - detune
q_f

## T1

In [ ]:
exp_cfg = {
    "dac": {
        # "res_pulse": "readout_rf",
        "res_pulse": "readout_dpm",
        "qub_pulse": {
            **DefaultCfg.get_pulse("pi_amp"),
            # "gain": 0.0,
        },
        # "reset": "pulse",
        # "reset_pulse": "reset_red",
        "reset": "mux_dual_pulse",
        "reset_pulse1": "mux_reset1",
        "reset_pulse2": "mux_reset2",
    },
    "adc": {
        "relax_delay": 0.0,  # us
        # "relax_delay": 3*t1,  # us
    },
}

In [ ]:
# exp_cfg["sweep"] = make_sweep(0.0, 50, 51)
exp_cfg["sweep"] = make_sweep(0.01, 5 * t1, 51)
cfg = make_cfg(exp_cfg, reps=1000, rounds=10)

# zs.visualize_t1(soccfg, cfg, time_fly=timeFly)
Ts, signals = zs.measure_t1(soc, soccfg, cfg)

In [ ]:
start = 1
stop = -1

%matplotlib inline
t1, _ = zf.T1_analyze(
    Ts[start:stop], signals[start:stop], max_contrast=True, dual_exp=False
)
t1

In [ ]:
filename = f"{qub_name}_t1@{cur_A * 1e3:.3f}mA"
save_data(
    filepath=os.path.join(database_path, filename),
    x_info={"name": "Time", "unit": "s", "values": Ts * 1e-6},
    z_info={"name": "Signal", "unit": "a.u.", "values": signals},
    comment=make_comment(cfg, f"t1 = {t1}us"),
    tag="TimeDomain/t1",
    server_ip=data_host,
)

## T2Echo

In [ ]:
exp_cfg = {
    "dac": {
        "res_pulse": "readout_dpm",
        "pi_pulse": "pi_amp",
        "pi2_pulse": "pi2_amp",
        # "reset": "pulse",
        # "reset_pulse": "reset_red",
        "reset": "mux_dual_pulse",
        "reset_pulse1": "mux_reset1",
        "reset_pulse2": "mux_reset2",
    },
    "adc": {
        "relax_delay": 0.5,  # us
        # "relax_delay": 3*t1,  # us
    },
}

In [ ]:
# exp_cfg["sweep"] = make_sweep(0.0, 1.5 * t2r, 101)
exp_cfg["sweep"] = make_sweep(0.0, 1.5 * t2e, 101)
cfg = make_cfg(exp_cfg, reps=1000, rounds=10)

activate_detune = 5.0 / (cfg["sweep"]["stop"] - cfg["sweep"]["start"])
print(f"activate_detune: {activate_detune:.2f}")
# zs.visualize_t2echo(soccfg, cfg, detune=activate_detune, time_fly=timeFly)
Ts, signals = zs.measure_t2echo(soc, soccfg, cfg, detune=activate_detune)

In [ ]:
%matplotlib inline
t2e, detune, _, _ = zf.T2fringe_analyze(Ts, signals, max_contrast=True)
# t2e, _ = zf.T2decay_analyze(Ts, signals, max_contrast=True)

In [ ]:
filename = f"{qub_name}_t2echo@{cur_A * 1e3:.3f}mA"
save_data(
    filepath=os.path.join(database_path, filename),
    x_info={"name": "Time", "unit": "s", "values": Ts * 1e-6},
    z_info={"name": "Signal", "unit": "a.u.", "values": signals},
    comment=make_comment(cfg, f"t2echo = {t2e}us"),
    tag="TimeDomain/t2echo",
    server_ip=data_host,
)

# Single shot

## Ground state & Excited state

In [ ]:
exp_cfg = {
    "dac": {
        # "res_pulse": "readout_rf",
        "res_pulse": {
            **DefaultCfg.get_pulse("readout_dpm"),
            # "length": 0.2 * t1 + timeFly + 0.1,
            # "ro_length": 0.2 * t1,
        },
        "qub_pulse": "pi_amp",
        # "qub_pulse": "reset_red",
        # "qub_pulse": {
        #     **DefaultCfg.get_pulse("pi_amp"),
        #     # "gain": 0.0,
        # },
        # "reset": "pulse",
        # "reset_pulse": "reset_red",
        "reset": "mux_dual_pulse",
        "reset_pulse1": "mux_reset1",
        "reset_pulse2": "mux_reset2",
    },
    "adc": {
        # "relax_delay": 5 * t1,  # us
        "relax_delay": 0.0,  # us
    },
}
exp_cfg["dac"]["res_pulse"]["ro_length"]

In [ ]:
cfg = make_cfg(exp_cfg, shots=1000000)

%matplotlib widget
signals = zs.measure_singleshot(soc, soccfg, cfg)

In [ ]:
%matplotlib inline
fid, _, _, pops = zf.singleshot_ge_analysis(signals, backend="pca")
print(f"Optimal fidelity after rotation = {fid:.1%}")

In [ ]:
n_gg, n_ge, n_eg, n_ee = pops
n_g = n_gg
n_e = n_ge
if n_e > n_g:
    n_g, n_e = n_e, n_g
n_g, n_e

In [ ]:
eff_T, err_T = zf.effective_temperature(population=[(n_g, 0.0), (n_e, q_f)], plot=False)
eff_T, err_T

In [ ]:
# filename = f"single_shot_rf_{qub_name}"
filename = f"{qub_name}_singleshot_ge@{cur_A * 1e3:.3f}mA"
save_data(
    filepath=os.path.join(database_path, filename),
    x_info={"name": "shot", "unit": "point", "values": np.arange(cfg["shots"])},
    z_info={"name": "Signal", "unit": "a.u.", "values": signals},
    y_info={"name": "ge", "unit": "", "values": np.array([0, 1])},
    comment=make_comment(
        cfg, f"fide: {fid:.1%}, (n_g, n_e): ({n_g:.1%}, {n_e:.1%}), eff_T: {eff_T:.1f}"
    ),
    tag="SingleShot/ge",
    server_ip=data_host,
)